In [16]:
import pandas as pd
import numpy as np
import copy

from tqdm import tqdm, trange

from datetime import datetime, timedelta, time

In [17]:
from solver import Solver

In [18]:
slvr = Solver('data/flights.json', 'data/crew.json')

slvr.flights

c:\Users\user\ML\Optimisation\crew-assignment\solver.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.flights['dep'] = pd.to_datetime(self.flights['dep']).dt.time
c:\Users\user\ML\Optimisation\crew-assignment\solver.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.flights['arr'] = pd.to_datetime(self.flights['arr']).dt.time


,day,id,origin,dest,dep,arr,duration,type
0,1,D1_F026,FRA,CPH,06:00:00,07:00:00,1,B737
1,1,D1_F043,DUB,CDG,06:00:00,07:00:00,1,B737
2,1,D1_F005,DUB,BCN,06:00:00,08:00:00,2,B737
3,1,D1_F023,FRA,CDG,06:00:00,08:00:00,2,B737
4,1,D1_F031,BCN,MXP,06:00:00,08:00:00,2,A320
...,...,...,...,...,...,...,...,...
245,5,D5_F042,LHR,DUB,18:00:00,19:00:00,1,A320
246,5,D5_F003,DUB,CPH,18:00:00,20:00:00,2,A320
247,5,D5_F001,MXP,CDG,19:00:00,20:00:00,1,A319
248,5,D5_F021,LHR,AMS,19:00:00,20:00:00,1,A320


In [19]:
slvr.crew

,id,name,role,base,qualified
0,C0,Joe Patterson,captain,AMS,"[B737, A320]"
1,C1,Kellie Lucas,captain,DUB,[B737]
2,C2,Joshua Green,captain,AMS,"[B737, A319]"
3,C3,Jeanette Novak,captain,MXP,"[B737, A320]"
4,C4,Amanda Mcdaniel,captain,AMS,[A320]
...,...,...,...,...,...
95,FO45,Jesse Williams,first_officer,MXP,[A319]
96,FO46,Jennifer Kelly,first_officer,DUB,[B737]
97,FO47,Jill Richmond,first_officer,LHR,[A320]
98,FO48,Antonio Gordon,first_officer,CPH,"[A320, A319]"


In [20]:
slvr.initial_assignment()

slvr.assignment

{'D1_F026': {'captain': np.str_('C48'),
  'first_officer': np.str_('FO28'),
  'dead_heading': []},
 'D1_F043': {'captain': np.str_('C46'),
  'first_officer': np.str_('FO35'),
  'dead_heading': []},
 'D1_F005': {'captain': np.str_('C44'),
  'first_officer': np.str_('FO26'),
  'dead_heading': []},
 'D1_F023': {'captain': np.str_('C48'),
  'first_officer': np.str_('FO12'),
  'dead_heading': []},
 'D1_F031': {'captain': np.str_('C32'),
  'first_officer': np.str_('FO16'),
  'dead_heading': []},
 'D1_F014': {'captain': np.str_('C14'),
  'first_officer': np.str_('FO10'),
  'dead_heading': []},
 'D1_F044': {'captain': np.str_('C24'),
  'first_officer': np.str_('FO17'),
  'dead_heading': []},
 'D1_F047': {'captain': np.str_('C12'),
  'first_officer': np.str_('FO30'),
  'dead_heading': []},
 'D1_F011': {'captain': np.str_('C49'),
  'first_officer': np.str_('FO2'),
  'dead_heading': []},
 'D1_F015': {'captain': None,
  'first_officer': np.str_('FO6'),
  'dead_heading': []},
 'D1_F024': {'captain'

In [21]:
slvr.initial_crew_state()
slvr.crew_state 

{'C0': [{'day': 4,
   'flight': 'D4_F014',
   'role': 'captain',
   'origin': 'AMS',
   'destination': 'DUB',
   'depart': datetime.time(7, 0),
   'arrive': datetime.time(8, 0)},
  {'day': 4,
   'flight': 'D4_F017',
   'role': 'captain',
   'origin': 'AMS',
   'destination': 'BCN',
   'depart': datetime.time(10, 0),
   'arrive': datetime.time(11, 0)},
  {'day': 4,
   'flight': 'D4_F021',
   'role': 'captain',
   'origin': 'AMS',
   'destination': 'MXP',
   'depart': datetime.time(10, 0),
   'arrive': datetime.time(11, 0)},
  {'day': 4,
   'flight': 'D4_F006',
   'role': 'captain',
   'origin': 'AMS',
   'destination': 'DUB',
   'depart': datetime.time(12, 0),
   'arrive': datetime.time(14, 0)}],
 'C1': [{'day': 1,
   'flight': 'D1_F010',
   'role': 'captain',
   'origin': 'DUB',
   'destination': 'AMS',
   'depart': datetime.time(15, 0),
   'arrive': datetime.time(16, 0)},
  {'day': 3,
   'flight': 'D3_F048',
   'role': 'captain',
   'origin': 'DUB',
   'destination': 'BCN',
   'depart

In [29]:
total_assignment_cost, diagnostics = slvr.compute_assignment_cost(slvr.assignment)
print(total_assignment_cost)

96100


In [30]:
diagnostics

{'unassigned_flights': 0,
 'base_mismatches': 192,
 'qualification_mismatches': 0,
 'deadheading_count': 1,
 'duplicate_roles': 0,
 'crew_usage': {np.str_('C14'): 5,
  np.str_('FO28'): 3,
  np.str_('C46'): 5,
  np.str_('FO35'): 3,
  'C1': 5,
  'FO2': 15,
  'C6': 14,
  'FO8': 4,
  np.str_('C32'): 3,
  np.str_('FO16'): 5,
  np.str_('FO10'): 5,
  np.str_('C24'): 7,
  np.str_('FO17'): 8,
  np.str_('C12'): 9,
  np.str_('FO30'): 7,
  'C34': 5,
  np.str_('FO12'): 6,
  np.str_('FO6'): 6,
  'C10': 9,
  'FO19': 3,
  np.str_('C45'): 11,
  np.str_('FO40'): 3,
  np.str_('C33'): 5,
  np.str_('FO32'): 4,
  'C44': 4,
  np.str_('FO1'): 14,
  'C35': 6,
  np.str_('FO9'): 6,
  'C2': 11,
  'FO15': 9,
  'FO0': 6,
  'FO38': 10,
  np.str_('C39'): 2,
  np.str_('FO20'): 5,
  np.str_('C11'): 6,
  'FO27': 6,
  np.str_('FO44'): 7,
  np.str_('C4'): 3,
  np.str_('C16'): 7,
  np.str_('C27'): 9,
  'C13': 5,
  'FO33': 3,
  'C29': 4,
  'FO29': 3,
  'FO43': 1,
  np.str_('C3'): 9,
  np.str_('C42'): 10,
  np.str_('FO21'): 

In [23]:
total_crew_cost, diagnostics = slvr.compute_crew_cost(slvr.crew_state)
print(total_crew_cost)

540300


In [24]:

num_destroy_operators = 2
num_repair_operators = 1



In [25]:
#print(f"{total_assignment_cost}, {total_crew_cost} => {total_assignment_cost+total_crew_cost}, \n {r_assignment_cost}, {r_crew_cost} => {r_assignment_cost+r_crew_cost}")

In [26]:
total_assignment_cost, _ = slvr.compute_assignment_cost(slvr.assignment)
total_crew_cost, _ = slvr.compute_crew_cost(slvr.crew_state)

for i in range(1000):

    destroy_repair_strength = np.random.choice([0.01, 0.05, 0.1, 0.2])
    num_to_destroy = int(len(slvr.flights) * destroy_repair_strength)

    d_assignment, d_crew_state, flights_destroyed, affected_crew_list = slvr.destroy(num_to_destroy)

    r_assignment, r_crew_state = slvr.repair()
    r_assignment_cost, _ = slvr.compute_assignment_cost(slvr.r_assignment)
    r_crew_cost, _ = slvr.compute_crew_cost(slvr.r_crew_state)

    if (r_assignment_cost+r_crew_cost) < (total_assignment_cost+total_crew_cost):
        slvr.accept_change()
        total_assignment_cost = r_assignment_cost
        total_crew_cost = r_crew_cost
    
        print(f"{total_assignment_cost}, {total_crew_cost} => {total_assignment_cost+total_crew_cost}, {num_to_destroy} destroy strength")

print("finished")
print(f"{total_assignment_cost}, {total_crew_cost} => {total_assignment_cost+total_crew_cost}")

66200, 514100 => 580300, 2 destroy strength
69700, 506300 => 576000, 25 destroy strength
94100, 466400 => 560500, 50 destroy strength
95100, 463900 => 559000, 2 destroy strength
95100, 462900 => 558000, 2 destroy strength
85600, 469500 => 555100, 12 destroy strength
86100, 457500 => 543600, 12 destroy strength
86600, 456100 => 542700, 2 destroy strength
87100, 453300 => 540400, 2 destroy strength
86600, 453300 => 539900, 12 destroy strength
87100, 452300 => 539400, 2 destroy strength
93600, 445400 => 539000, 12 destroy strength
91100, 445000 => 536100, 25 destroy strength
91600, 443800 => 535400, 2 destroy strength
91600, 442100 => 533700, 12 destroy strength
91600, 441800 => 533400, 25 destroy strength
91100, 441300 => 532400, 2 destroy strength
91600, 440300 => 531900, 50 destroy strength
91600, 438800 => 530400, 2 destroy strength
92600, 437500 => 530100, 2 destroy strength
94100, 435700 => 529800, 2 destroy strength
95600, 433700 => 529300, 2 destroy strength
94600, 434500 => 52910

In [27]:
slvr.assignment

{'D1_F026': {'captain': np.str_('C14'),
  'first_officer': np.str_('FO28'),
  'dead_heading': []},
 'D1_F043': {'captain': np.str_('C46'),
  'first_officer': np.str_('FO35'),
  'dead_heading': []},
 'D1_F005': {'captain': 'C1', 'first_officer': 'FO2', 'dead_heading': []},
 'D1_F023': {'captain': 'C6', 'first_officer': 'FO8', 'dead_heading': []},
 'D1_F031': {'captain': np.str_('C32'),
  'first_officer': np.str_('FO16'),
  'dead_heading': []},
 'D1_F014': {'captain': np.str_('C14'),
  'first_officer': np.str_('FO10'),
  'dead_heading': []},
 'D1_F044': {'captain': np.str_('C24'),
  'first_officer': np.str_('FO17'),
  'dead_heading': []},
 'D1_F047': {'captain': np.str_('C12'),
  'first_officer': np.str_('FO30'),
  'dead_heading': []},
 'D1_F011': {'captain': 'C34',
  'first_officer': np.str_('FO12'),
  'dead_heading': []},
 'D1_F015': {'captain': 'C6',
  'first_officer': np.str_('FO6'),
  'dead_heading': []},
 'D1_F024': {'captain': 'C10', 'first_officer': 'FO19', 'dead_heading': []},
 

In [28]:
slvr.crew_state

{'C0': [{'day': 3,
   'flight': 'D3_F001',
   'role': 'captain',
   'origin': 'AMS',
   'destination': 'LHR',
   'depart': datetime.time(11, 0),
   'arrive': datetime.time(13, 0),
   'duration': 2},
  {'day': 4,
   'flight': 'D4_F014',
   'role': 'captain',
   'origin': 'AMS',
   'destination': 'DUB',
   'depart': datetime.time(7, 0),
   'arrive': datetime.time(8, 0)},
  {'day': 4,
   'flight': 'D4_F037',
   'role': 'captain',
   'origin': 'DUB',
   'destination': 'CDG',
   'depart': datetime.time(11, 0),
   'arrive': datetime.time(12, 0),
   'duration': 1},
  {'day': 4,
   'flight': 'D4_F023',
   'role': 'captain',
   'origin': 'CDG',
   'destination': 'CPH',
   'depart': datetime.time(16, 0),
   'arrive': datetime.time(18, 0),
   'duration': 2},
  {'day': 5,
   'flight': 'D5_F037',
   'role': 'captain',
   'origin': 'BCN',
   'destination': 'DUB',
   'depart': datetime.time(8, 0),
   'arrive': datetime.time(10, 0),
   'duration': 2},
  {'day': 5,
   'flight': 'D5_F043',
   'role': 'c